<a href="https://colab.research.google.com/github/Mohi294/Article-Classification/blob/main/topic_classification_LSTm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive,files


drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
import tensorflow as tf
from tensorflow.keras.layers import Bidirectional,LSTM,TextVectorization,Dense,Embedding,Dropout,BatchNormalization,GRU,Attention
import pandas as pd
import numpy as np


reading data

In [3]:
df = pd.read_csv(r'/content/gdrive/My Drive/finalfinal.csv')


In [4]:
df

,Unnamed: 0,FileName,CategoryId,CategoryName,WordCount,Language,text
0,0,1307.pdf,2,کامپیوتر و شبکه,2479,en,mrs n v l indira dr r kiran kumar internationa...
1,1,1308.pdf,2,کامپیوتر و شبکه,1350,en,cseit received dec accepted dec november decem...
2,2,1309.pdf,2,کامپیوتر و شبکه,1453,en,scalable recommender systems machine learning ...
3,3,1310.pdf,2,کامپیوتر و شبکه,5855,en,counterfactual estimation optimization click m...
4,4,1320.pdf,2,کامپیوتر و شبکه,9002,en,turun kauppakorkeakoulu • turku school economi...
...,...,...,...,...,...,...,...
56619,56619,189993.docx,20,عمومی,3524,fa,آیا ایرانیان روزی انتقام خود را از حمله خانمان...
56620,56620,189994.docx,49,مهندسی صنایع,14367,fa,doi ier مقاله منتخب هفدهمین کنفرانس بین المللی...
56621,56621,189995.docx,20,عمومی,1584,en,story valentine’s day valentine day also calle...
56622,56622,190004.docx,20,عمومی,1773,en,us failed stop oil exports iran iran said satu...


In [5]:
df = df[(df['text'].notna()) & (df['Language'] == 'en')]

x = df['text']
y = df['CategoryId']

splitting test and train

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.1, random_state = 0, shuffle = True)

train_dataset = pd.concat([X_train,y_train],axis=1)
test_dataset = pd.concat([X_test,y_test],axis=1) 


In [7]:
from sklearn.preprocessing import LabelEncoder
y_train = LabelEncoder().fit_transform(y_train)
y_test = LabelEncoder().fit_transform(y_test)

# from tensorflow.keras.utils import to_categorical

# y_train =to_categorical(y_train)
# y_test = to_categorical(y_test)

building vocabulary

In [8]:
VOCAB_SIZE = 100000

encoder = TextVectorization(max_tokens=VOCAB_SIZE)
encoder.adapt(train_dataset["text"].to_numpy().reshape(-1,1))

In [9]:
vocab = np.array(encoder.get_vocabulary())

vocab

array(['', '[UNK]', '–', ..., 'out\xad', 'outcasts', 'osteoclast'],
      dtype='<U797')

In [10]:
#train_dataset["text"].to_numpy().reshape(-1,1)

building model architecture

In [11]:
model = tf.keras.Sequential([
    tf.keras.Input(shape=(1,), dtype=tf.string),
    encoder,
    Embedding(
        input_dim=len(encoder.get_vocabulary()),
        output_dim=64,
        mask_zero=True),
    Bidirectional(LSTM(200,return_sequences=True)),
    Dropout(0.4),
    BatchNormalization(),
    Bidirectional(LSTM(100)),
    Dropout(0.2),
    BatchNormalization(),
    Dense(64, activation='relu',kernel_regularizer='l2'),
    Dense(58,activation='softmax')
])

In [12]:
# model = tf.keras.Sequential([
#     tf.keras.Input(shape=(1,), dtype=tf.string),
#     encoder,
#     Embedding(
#         input_dim=len(encoder.get_vocabulary()),
#         output_dim=120,
#         mask_zero=True),
#     # Bidirectional(LSTM(200,return_sequences=True)),
#     # Dropout(0.4),
#     # BatchNormalization(),
#     Bidirectional(LSTM(200, return_sequences=False)),
#     Dropout(0.5),
#     BatchNormalization(),
#     Dense(48, activation='relu',kernel_regularizer='l1'),
#     Dense(58,activation='softmax')
    
# ])

In [13]:
#callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)

compiling model

In [14]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=tf.keras.optimizers.Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False),
              metrics=['accuracy'])

/usr/local/lib/python3.7/dist-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [ ]:
history = model.fit(X_train,y_train,batch_size=100,epochs=27,validation_data=(X_test,y_test))

Epoch 1/27


In [ ]:
test_loss, test_acc = model.evaluate(X_test,y_test)

print('Test Loss:', test_loss)
print('Test Accuracy:', test_acc)

plotting model results

In [ ]:
from matplotlib import pyplot

pyplot.plot(history.history['loss'])
pyplot.plot(history.history['val_loss'])
pyplot.title('model train vs validation loss')
pyplot.ylabel('loss')
pyplot.xlabel('epoch')
pyplot.legend(['train', 'validation'], loc='upper right')
pyplot.show()

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(16, 8))
#plt.subplot(1, 2, 1)
plt.plot(model.history.history['loss'] , label='train')
plt.plot(model.history.history['val_loss'], label='val')
#plt.ylim(None, 1)
plt.legend()


In [ ]:


#plt.subplot(1, 2, 2)
plt.plot(model.history.history['accuracy'] , label='train')
plt.plot(model.history.history['val_accuracy'], label='val')
plt.ylim(0, None)
plt.legend()

In [ ]:
model.history